Guide: sections split by algorithm type (NLP (that aren't purely text embedders), LLM, text embedding), sub-sections are the particular model, sub-sub-sections is 1 for getting the scores and 2 for evaluation metrics

**0) PILOT DATA SETUP**

In [ ]:
raw_data = open("Pain point pilot data- ground truth.txt", "r") 
data = raw_data.read() 

data_list = data.split("\t") 
#print(data_list) #if u wanna compare raw to the testable data

#remove extra characters
for string in data_list:
    if "\n" in string:
        new_string = string.replace("\n", "")
        #print(string)
        i = data_list.index(string)
        data_list[i] = new_string
    if '"' in string:
        new_string = string.replace('"', "")
        i = data_list.index(string)
        data_list[i] = new_string

#print(data_list) 
raw_data.close() 

testable_data = []
ground_truths = []
count = 0
for string in data_list:
    if (len(string) > 2):
        testable_data.append(string)
        count += 1
    elif (len(string) > 0):
        ground_truths.append(string)

print(len(testable_data)) #for reference
print(testable_data)
print(len(ground_truths))
print(ground_truths)

codes = ["Lathe chuck overtightend", "18 V Drills too large to use with one hand", "Disposable gloves are a large size", "Miter saw is on tall table (awkward to use)", "People forget to put away the clamps", "Someone left sawdust and wood chips everywhere", "Someone squeezed through aisle and bumped user (bumped e-stop)", "Wood scraps too small to be useful", "Digging through the unlabeled cabinets looking for drill", "Trash bag is ripped", "Had to wait for epoxy to cure"]
print(codes) #for reference


46
['cutting wood', 'didn’t know how to use lathe', 'Finding drill', 'Taking out trash', 'Finding clamp', 'Loosening the lathe', 'Cleaning up after others', 'Too small of wood scraps.', 'Had to take trash out.', 'Had to find a clamp.', 'Had to dig in unlabeled container for a 12v drill.', 'Got bumped while using sander, accidentally pressed emergency stop button.', 'Had to clean off messy workspace.', 'People putting garbage in the scrp bin', 'People not throwing trash properly ripping the bag', 'People not putting things where they go', 'People not putting things back the way they were found', 'People leaving a mess and not cleaning up', 'bumped by person using belt sander', 'garbage bag had hole', 'sawdust left on machine', 'lathe chuck tightened too tight', 'only found 18V drill which was too big', 'Someone pushed them ', 'Finding clamps', 'Loosening the chuck', 'people squeezing by and caused him to restart', 'finding a drill', 'clearing the scrap bin', 'Height of miter saw', 'Tras

In [3]:
#process into a dataset 
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

data_numbers = []
for i in range(len(testable_data)):
     data_numbers.append(i)

data = {
    'TD': data_numbers,
    'text': testable_data
}
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['TD', 'text'],
    num_rows: 46
})



**1.1.1) NLP (BART thematic analysis)** (https://huggingface.co/facebook/bart-large-mnli)

In [4]:
#model set-up
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Device set to use cuda:0


In [ ]:
import sys

original_stdout = sys.stdout

with open('output-BART.txt', 'w') as f:
    sys.stdout = f

    for i in range(5): 
        print("\n")
        print("DATA: " + testable_data[i] + "\n")
        #sequence_to_classify = testable_data[i]
        sequence_to_classify = KeyDataset(dataset, "text")[i]
        for code in codes:
            candidate_labels = code
            results = classifier(sequence_to_classify, candidate_labels, multi_label=False) #should I keep multi_labels=True? (whats the diff)
            print(str(results["scores"][0]) + "\t" + code) #only gimme the scores

f.close()

In [6]:
#testing w/ shorter codes
sequence_to_classify = "Finding drill"
candidate_labels = ["Screwdriver", "cat", "hammer", "look", "practice"]
results = classifier(sequence_to_classify, candidate_labels, multi_label=False) #should I keep multi_labels=True? (whats the diff)
print(results)

#try homonyms w/ other models

{'sequence': 'Finding drill', 'labels': ['practice', 'Screwdriver', 'look', 'hammer', 'cat'], 'scores': [0.8722097277641296, 0.0930788442492485, 0.017568174749612808, 0.00991278886795044, 0.0072304424829781055]}


**1.1.2) BART evaluation w/ confusion matrix**

In [9]:
#model set-up
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


predictions = []
for i in range(len(testable_data)): 
    max_score = 0;
    idx_of_max = 0;
    #sequence_to_classify = testable_data[i]
    sequence_to_classify = KeyDataset(dataset, "text")[i]
    for j in range(len(codes)):
        candidate_labels = codes[j]
        results = classifier(sequence_to_classify, candidate_labels, multi_label=False) #should I keep multi_labels=True? (whats the diff)
        if (results["scores"][0] > max_score):
            idx_of_max = j+1
            max_score = results["scores"][0]
    predictions.append(idx_of_max)

predictions_str = []
for pred in predictions:
    str_pred = str(pred)
    predictions_str.append(str_pred)
print((predictions_str))
print((ground_truths))

Device set to use cuda:0


['1', '8', '9', '3', '1', '1', '3', '8', '6', '4', '9', '11', '6', '8', '10', '5', '10', '6', '2', '10', '1', '1', '2', '7', '2', '8', '7', '9', '8', '4', '8', '10', '11', '8', '11', '6', '7', '11', '5', '9', '3', '7', '6', '2', '4', '5']
['8', '0', '9', '10', '9', '1', '6', '8', '0', '5', '9', '7', '6', '0', '10', '5', '5', '6', '7', '10', '6', '1', '2', '7', '5', '1', '7', '9', '8', '4', '0', '10', '11', '5', '9', '6', '7', '5', '7', '9', '3', '7', '6', '2', '4', '5']


In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

print(confusion_matrix(ground_truths, predictions_str, labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]))
BART_eval = f1_score(ground_truths, predictions_str, average=None) 
print(BART_eval)
#check what we wanna do for the average parameter (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

[[0 0 0 0 0 0 1 0 3 0 0 0]
 [0 2 0 0 0 0 0 0 1 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 1 0 1 2 0 0 1 0 1 1]
 [0 1 0 1 0 0 4 0 0 0 0 0]
 [0 0 1 0 0 1 0 4 0 0 0 1]
 [0 1 0 0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0 4 0 1]
 [0 0 0 1 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]]
[0.         0.5        0.75       0.4        0.66666667 0.5
 0.8        0.4        0.72727273 0.72727273 0.4        0.8       ]


**1.2.1) BERT sentence similarity**

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("tomaarsen/static-similarity-mrl-multilingual-v1")
# Run inference
sentences = [
    'It is known for its dry red chili powder .',
    'It is popular for dry red chili powder .',
    'These monsters will move in large groups .',
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 1024]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]


Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\venv\Lib\site-packages\IPython\core\history.py", line 853, in writeout_cache
    self._writeout_input_cache(conn)
  File "C:\ProgramData\anaconda3\envs\venv\Lib\site-packages\IPython\core\history.py", line 836, in _writeout_input_cache
    conn.execute("INSERT INTO history VALUES (?, ?, ?, ?)",
sqlite3.IntegrityError: UNIQUE constraint failed: history.session, history.line

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\venv\Lib\site-packages\IPython\core\history.py", line 908, in run
    self.history_manager.writeout_cache(self.db)
  File "C:\ProgramData\anaconda3\envs\venv\Lib\site-packages\decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\envs\venv\Lib\site-pack

**2.1) LLM: Microsoft Phi** (https://huggingface.co/microsoft/Phi-4-mini-instruct)
- it works fine on a singular case, just loses efficiency when it has to rerun every time.
- can be fixed with datasets (?), so GPU is optimized

In [33]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "calculate the percent similarity between 'Finding drill' and 'drill'. Don't tell me how, just give me the percent similarity."}
]
pipe = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", trust_remote_code=True)
print(pipe(messages))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': "calculate the percent similarity between 'Finding drill' and 'drill'. Don't tell me how, just give me the percent similarity."}, {'role': 'assistant', 'content': "The percent similarity between 'Finding drill' and 'drill' is approximately 83.33%."}]}]


In [23]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import sys


#original_stdout = sys.stdout
#with open('output.txt', 'w') as f:
#    sys.stdout = f

pipe = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", trust_remote_code=True)
for i in range(2): 
    print("\n")
    print("DATA: " + testable_data[i] + "\n")
    for code in codes:
        messages = [
            {"role": "user", "content": "calculate the percent similarity between " + KeyDataset(dataset, "text")[i] + " and " + code + ". Don't tell me how, just give me the percent similarity."}
        ] 
        result = pipe(messages)
        print(result[0]["generated_text"][1]["content"])

#f.close()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0




DATA: cutting wood

The percent similarity between cutting wood and a lathe chuck overtightened is approximately 0%. These
The percent similarity between cutting wood and 18 V drills too large to use with one hand is approximately
0%
The percent similarity between cutting wood and using a miter saw on a tall table (awkward to
The percent similarity between cutting wood and people forgetting to put away clamps is approximately 0%. These activities
The percent similarity between cutting wood and someone leaving sawdust and wood chips everywhere is approximately 95%.
The percent similarity between cutting wood and someone squeezing through an aisle and bumping an e-stop is approximately
Approximately 95%
The percent similarity between cutting wood and digging through unlabeled cabinets looking for a drill is approximately 25
The percent similarity between cutting wood and a trash bag being ripped is approximately 0%. These actions are
The percent similarity between cutting wood and havi

In [2]:
# trying a yes/no prompt
from transformers import pipeline

messages = [
    {"role": "user", "content": "Is there any similarity between 'Finding drill' and 'drill'? If so, tell me the percent similarity."}
]
pipe = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", trust_remote_code=True)
print(pipe(messages))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': "Is there any similarity between 'Finding drill' and 'drill'? If so, tell me the percent similarity."}, {'role': 'assistant', 'content': 'The words "Finding drill" and "drill" share some similarities, but they are not identical'}]}]


**2.2) LLM: Meta Llama**
- need to refine prompts

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
for i in range(2): 
    print("\n")
    print("DATA: " + testable_data[i] + "\n")
    for code in codes:
        messages = [
            {"role": "user", "content": "calculate the percent similarity between " + testable_data[i] + " and " + code + ". Don't tell me how, just give me the percent similarity."}
        ] 
        result = pipe(messages)
        print(result[0]["generated_text"][1]["content"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




DATA: cutting wood



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I cannot verify the percent similarity between cutting wood and lathe chuck overtightened.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the given phrases, I would estimate the percent similarity to be around 0%. They seem


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To calculate the percent similarity, I'll compare the two phrases:

1. "didn’t know how


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To calculate the percent similarity, I'll use a simple algorithm. 

After analyzing the two phrases,
To calculate the percent similarity, I'll compare the two phrases.

Here are the phrases with their individual


In [15]:
# trying a yes/no prompt
messages = [
    {"role": "user", "content": "Pretend you are the world's best thematic analyst. Is there any similarity between 'Finding drill' and 'drill'? If so, tell me the percent similarity. Don't tell me how, just give me the percentage."},
]
pipe(messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user',
    'content': "Pretend you are the world's best thematic analyst. Is there any similarity between 'Finding drill' and 'drill'? If so, tell me the percent similarity. Don't tell me how, just give me the percentage."},
   {'role': 'assistant',
    'content': "The similarity between 'Finding drill' and 'drill' is approximately 72%."}]}]

**2.3) LLM: DeepSeek**
- needs to be done from terminal
- wait for HF to develop (LOL) or try vLLM

**3.1.1) text embedding: MPNet** (https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#semantic-search-models) 
- this is the pretrained model (general purpose), there are more on HF that might have more relevant training we can try
- runs super quickly (yay!) (faster than other 2)
- not the same as from the 2024 paper, might still contact the author for that

In [5]:
#full matrix to demo how text embedding works

from sentence_transformers import SentenceTransformer

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode([
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
])
similarities = model.similarity(embeddings, embeddings)
print(similarities)


tensor([[1.0000, 0.6817, 0.0492],
        [0.6817, 1.0000, 0.0421],
        [0.0492, 0.0421, 1.0000]])


In [2]:
#how it can be used as a coder

from sentence_transformers import SentenceTransformer

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")
user_embeddings = model.encode("Loosening the lathe") #user response
code_embeddings = model.encode([
    "Someone squeezed through aisle and bumped user (bumped e-stop)", #code
    "Lathe chuck overtightend", #code
])
similarities = model.similarity(user_embeddings, code_embeddings)
print(similarities)

tensor([[0.0868, 0.6279]])


In [ ]:
from sentence_transformers import SentenceTransformer
import sys

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")

with open('output-MPNet.txt', 'w') as f:
    sys.stdout = f
    
    for i in range(5): 
        print("\n")
        print("DATA: " + testable_data[i] + "\n")
        user_embeddings = model.encode(testable_data[i])
        code_embeddings = model.encode(codes)
        similarities = model.similarity(user_embeddings, code_embeddings)
        results = similarities.tolist()
        for i in range(len(codes)): 
            print(str(results[0][i]) + "\t" + codes[i]) 

f.close()

**3.1.2) MPNet evaluation**

In [11]:
from sentence_transformers import SentenceTransformer
import sys

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")
predictions = []
 
for i in range(len(testable_data)): 
    max_score = 0;
    idx_of_max = 0;
    user_embeddings = model.encode(testable_data[i])
    code_embeddings = model.encode(codes)
    similarities = model.similarity(user_embeddings, code_embeddings)
    results = similarities.tolist()
    for j in range(len(codes)): 
        #print(results)
        if (results[0][j] > max_score):
            idx_of_max = j+1
            max_score = results[0][j]
    predictions.append(idx_of_max)

predictions_str = []
for pred in predictions:
    str_pred = str(pred)
    predictions_str.append(str_pred)
print((predictions_str))
print((ground_truths))

['8', '1', '9', '10', '5', '1', '7', '8', '10', '5', '9', '7', '6', '10', '10', '5', '6', '6', '7', '10', '6', '1', '2', '7', '5', '1', '7', '9', '8', '4', '10', '10', '11', '5', '9', '1', '7', '5', '4', '9', '3', '7', '6', '2', '4', '5']
['8', '0', '9', '10', '9', '1', '6', '8', '0', '5', '9', '7', '6', '0', '10', '5', '5', '6', '7', '10', '6', '1', '2', '7', '5', '1', '7', '9', '8', '4', '0', '10', '11', '5', '9', '6', '7', '5', '7', '9', '3', '7', '6', '2', '4', '5']


In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

print(confusion_matrix(ground_truths, predictions_str, labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]))
MPNet_eval = f1_score(ground_truths, predictions_str, average=None)
print(MPNet_eval)
#check what we wanna do for the average parameter (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

[[0 1 0 0 0 0 0 0 0 0 3 0]
 [0 3 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 6 1 0 0 0 0 0]
 [0 1 0 0 0 0 4 1 0 0 0 0]
 [0 0 0 0 1 0 0 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 1 0 0 0 5 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]]
[0.         0.75       0.72727273 1.         1.         1.
 0.8        0.85714286 0.72727273 0.85714286 1.         0.90909091]


**3.2.1) Text embedding: Jina Embeddings V2** (https://huggingface.co/jinaai/jina-embeddings-v2-base-en?library=sentence-transformers)
- same as in the physics ed paper

In [2]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['Had to find a clamp', 'People forget to put away the clamps'])
print(cos_sim(embeddings[0], embeddings[1]))
#this is a better way to use the model :)

0.81299573


In [6]:
#exact same formatting as MPNet, but this is the auto-generated transformers code, see above for a better version
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("jinaai/jina-embeddings-v2-base-en", trust_remote_code=True)

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.8468, 0.6408],
        [0.8468, 1.0000, 0.6417],
        [0.6408, 0.6417, 1.0000]])


In [ ]:
import sys
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method
code_embeddings = model.encode(codes)

results = []
with open('output-JinaEmbeddingsV2.txt', 'w') as f:
    sys.stdout = f
    
    for i in range(5): 
        print("\n")
        print("DATA: " + testable_data[i] + "\n")
        user_embeddings = model.encode(testable_data[i])
        code_embeddings = model.encode(codes)
        for k in range(len(codes)):
            results.append(cos_sim(user_embeddings, code_embeddings[k]))
        for i in range(len(codes)): 
            print(str(results[i]) + "\t" + codes[i]) 
        results = []
        

f.close()

**3.1.2) Jina Embeddings V2 evaluation**

In [13]:
import sys
from transformers import AutoModel
from numpy.linalg import norm

predictions = []
cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method
code_embeddings = model.encode(codes)

results = []
for i in range(len(testable_data)): 
    max_score = 0;
    idx_of_max = 0;
    user_embeddings = model.encode(testable_data[i])
    for k in range(len(codes)):
        results.append(cos_sim(user_embeddings, code_embeddings[k]))
    for j in range(len(codes)): 
        #print(results)
        if (results[j] > max_score):
            idx_of_max = j+1
            max_score = results[j]
    predictions.append(idx_of_max)
    results = []

predictions_str = []
for pred in predictions:
    str_pred = str(pred)
    predictions_str.append(str_pred)
print((predictions_str))
print((ground_truths))

['8', '1', '9', '10', '5', '1', '6', '8', '10', '5', '9', '7', '6', '10', '10', '5', '5', '5', '7', '10', '6', '1', '2', '7', '5', '1', '7', '9', '10', '4', '10', '10', '11', '5', '9', '1', '7', '5', '5', '9', '9', '7', '6', '2', '4', '5']
['8', '0', '9', '10', '9', '1', '6', '8', '0', '5', '9', '7', '6', '0', '10', '5', '5', '6', '7', '10', '6', '1', '2', '7', '5', '1', '7', '9', '8', '4', '0', '10', '11', '5', '9', '6', '7', '5', '7', '9', '3', '7', '6', '2', '4', '5']


In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

print(confusion_matrix(ground_truths, predictions_str, labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]))
JinaEmbeddingsV2_eval = f1_score(ground_truths, predictions_str, average=None)
print(JinaEmbeddingsV2_eval)
#check what we wanna do for the average parameter (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

[[0 1 0 0 0 0 0 0 0 0 3 0]
 [0 3 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 7 0 0 0 0 0 0]
 [0 1 0 0 0 1 4 0 0 0 0 0]
 [0 0 0 0 0 1 0 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 1 0]
 [0 0 0 0 0 1 0 0 0 5 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]]
[0.         0.75       0.66666667 1.         1.         0.
 1.         0.82352941 0.8        0.92307692 0.8        0.83333333]


**4) Formatting to compare evaluation results**
- so far only comparing F1 scores for each code

In [ ]:
with open('eval-results.txt', 'w') as f:
    sys.stdout = f

    print("**The Code, followed by the F1 score for BART, then MPNet, then Jina Embeddings V2**")
    print("\n")

    codes = ["Lathe chuck overtightend", "18 V Drills too large to use with one hand", "Disposable gloves are a large size", "Miter saw is on tall table (awkward to use)", "People forget to put away the clamps", "Someone left sawdust and wood chips everywhere", "Someone squeezed through aisle and bumped user (bumped e-stop)", "Wood scraps too small to be useful", "Digging through the unlabeled cabinets looking for drill", "Trash bag is ripped", "Had to wait for epoxy to cure"]
    codes.insert(0, "No code assigned")
    for i in range(len(codes)):
        print(codes[i] + "\t")
        print(BART_eval[i])
        print("\t")
        print(MPNet_eval[i])
        print("\t")
        print(JinaEmbeddingsV2_eval[i])
        print("\n")

f.close()